# Out-of-Distribution Detection

In [1]:
import os
import sys
import torch

current_dir = os.getcwd()
openood_dir = os.path.abspath(os.path.join(current_dir, '../OpenOOD/')) 

if openood_dir not in sys.path:
    sys.path.append(openood_dir)

from openood.networks import ResNet18_32x32
from openood.evaluation_api import Evaluator

In [2]:
# check MPS
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend")
else:
    device = torch.device("cpu")
    print("Using CPU backend")

Using MPS backend


In [3]:
net = ResNet18_32x32(num_classes=100) # num_class=100 (CIFAR-100)
net.load_state_dict(
    torch.load('cifar100_resnet18_32x32_base_e100_lr0.1_default/s0/best.ckpt', map_location=device)
)
net.to(device)
net.eval();

In [4]:
postprocessor_name = "knn"

In [5]:
evaluator = Evaluator(
    net,
    id_name='cifar100',                     # the target ID dataset
    data_root='./data',                    # change if necessary
    config_root=None,                      # see notes above
    preprocessor=None,                     # default preprocessing for the target ID dataset
    postprocessor_name=postprocessor_name, # the postprocessor to use
    postprocessor=None,                    # if you want to use your own postprocessor
    batch_size=200,                        # for certain methods the results can be slightly affected by batch size
    shuffle=False,
    num_workers=2)                         # could use more num_workers outside colab

Setup: 100%|██████████| 250/250 [00:43<00:00,  5.78it/s]


Starting automatic parameter search...


  0%|          | 0/5 [00:00<?, ?it/s]

: 